# 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 필요 라이브러리 불러오기

In [ ]:
# konlpy 설치
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 450kB 37.4MB/s 
     |████████████████████████████████| 92kB 7.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from konlpy.tag import Okt, Hannanum, Kkma,Komoran,Mecab
from sklearn.feature_extraction.text import TfidfVectorizer
from matplotlib import pyplot as plt
from datetime import datetime as dt


In [ ]:
# matplotlib 한글 폰트 출력코드
# 출처 : 데이터공방( https://kiddwannabe.blog.me)

import matplotlib
from matplotlib import font_manager, rc
import platform

try : 
    if platform.system() == 'Windows':
    # 윈도우인 경우
        font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
        rc('font', family=font_name)
    else:    
    # Mac 인 경우
        rc('font', family='AppleGothic')
except : 
    pass
matplotlib.rcParams['axes.unicode_minus'] = False   

In [ ]:
# 수지님 데이터 불러오는 부분
sj_path = '/content/drive/MyDrive/AniHelp_full_feat2.csv'
df = pd.read_csv(sj_path, index_col=0)
df.head()

,공고번호,축종,품종,털색,성별,중성화여부,특징,접수일시,구조사유,발생장소,공고기간,관할보호센터명,보호장소,관할기관,상태,성격,건강,나이
0,경북-예천-2018-00003,개,믹스견,NaN,암컷,아니오,NaN,2018-01-01,NaN,예천읍 통명리,2018-01-03 ~ 2018-01-17,예천군유기동물보호소,"경상북도 예천군 예천읍 충효로 632-84 (예천읍, 유기동물보호소)",경상북도 예천군,NaN,0,0,0
1,광주-남구-2018-00001,개,믹스견,흰색,수컷,미상,"18-0002콩지,다리부상",2018-01-01,NaN,남구 송하동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(안락사),0,2,0
2,광주-남구-2018-00002,개,믹스견,흰색,암컷,미상,18-0003콩이,2018-01-01,NaN,남구 송하동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(입양),0,0,0
3,광주-광산-2018-00001,개,보스턴 테리어,검흰,수컷,아니오,"18-0001볼타,스파이크 목줄착용",2018-01-01,NaN,광산구 하남동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 광산구,종료(자연사),0,0,0
4,경기-수원-2018-00001,개,믹스견,갈색,암컷,아니오,"교통사고 = 골반 / 척추 분리 골절 , 대퇴골두 골절 = 진통제 처치",2018-01-01,NaN,세류사거리,2018-01-01 ~ 2018-01-08,광교산동물병원,경기도 수원시 장안구 영화동 127-39,경기도 수원시,종료(안락사),0,2,0


In [ ]:
# 정희 데이터 불러오는 부분
jeonghee_path = '/content/drive/MyDrive/TwoPs/AniHelp_full_feat2.csv'
df = pd.read_csv(jeonghee_path, index_col=0)
df.head()

,공고번호,축종,품종,털색,성별,중성화여부,특징,접수일시,구조사유,발생장소,공고기간,관할보호센터명,보호장소,관할기관,상태,성격,건강,나이
0,경북-예천-2018-00003,개,믹스견,NaN,암컷,아니오,NaN,2018-01-01,NaN,예천읍 통명리,2018-01-03 ~ 2018-01-17,예천군유기동물보호소,"경상북도 예천군 예천읍 충효로 632-84 (예천읍, 유기동물보호소)",경상북도 예천군,NaN,0,0,0
1,광주-남구-2018-00001,개,믹스견,흰색,수컷,미상,"18-0002콩지,다리부상",2018-01-01,NaN,남구 송하동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(안락사),0,2,0
2,광주-남구-2018-00002,개,믹스견,흰색,암컷,미상,18-0003콩이,2018-01-01,NaN,남구 송하동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(입양),0,0,0
3,광주-광산-2018-00001,개,보스턴 테리어,검흰,수컷,아니오,"18-0001볼타,스파이크 목줄착용",2018-01-01,NaN,광산구 하남동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 광산구,종료(자연사),0,0,0
4,경기-수원-2018-00001,개,믹스견,갈색,암컷,아니오,"교통사고 = 골반 / 척추 분리 골절 , 대퇴골두 골절 = 진통제 처치",2018-01-01,NaN,세류사거리,2018-01-01 ~ 2018-01-08,광교산동물병원,경기도 수원시 장안구 영화동 127-39,경기도 수원시,종료(안락사),0,2,0


# 데이터 전처리

In [ ]:
# df['성별'].unique()
df[(df['축종']== '개') & (df['성별']=='미상')]

,공고번호,축종,품종,털색,성별,중성화여부,특징,접수일시,구조사유,발생장소,공고기간,관할보호센터명,보호장소,관할기관,상태,성격,건강,나이
55,전남-영광-2018-00001,개,믹스견,아이보리,미상,미상,어미가 없음,2018-01-01,강아지 9마리가 추위에 떨고 있음,불갑면 건무리,2018-01-01 ~ 2018-01-11,불갑동물임시보호소,전라남도 영광군 불갑면 함영로 2758,전라남도 영광군,종료(입양),0,0,0
163,전남-나주-2018-00003,개,푸들,갈색,미상,미상,암1수1,2018-01-01,NaN,우산리폐교,2018-01-05 ~ 2018-01-15,빛고을애견,전라남도 나주시 남평읍 회재로 22,전라남도 나주시,종료(안락사),0,0,0
172,울산-남구-2018-00002,개,말티즈,흰색,미상,미상,사나움 경계심있음 동물정보추후예정,2018-01-01,NaN,에코하이츠105동,2018-01-03 ~ 2018-01-15,울산유기동물보호센터,울산광역시 울주군 온양읍 화산발리로 543 (온양읍),울산광역시 남구,종료(기증),1,0,0
383,전남-장성-2018-00001,개,기타,흰색,미상,미상,흰색,2018-01-02,NaN,장성읍 일대,2018-01-02 ~ 2018-01-12,우남동물병원,전라남도 장성군 장성읍 구암길 46-24 (장성읍),전라남도 장성군,종료(입양),0,0,0
513,경남-남해-2018-00002,개,믹스견,"흰색 3두, 얼룩 1두, 검정 1두",미상,아니오,"한 어미에서 나온 새끼, 온순함, 사람을 잘 따름, 암컷 3두, 수컷 2두",2018-01-02,유기동물 신고,남해군 남면 선구리 선구마을 내,2018-01-03 ~ 2018-01-15,남해군유기동물보호소,경상남도 남해군 이동면 무림로 33-13 (이동면),경상남도 남해군,종료(입양),2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434111,경북-예천-2021-00176,개,믹스견,흰색 갈색,미상,미상,"사람 좋아하고 조용함, 입양전제로 임보중.",2021-06-22,민원신고,군청당직실,2021-06-22 ~ 2021-07-02,예천군유기동물보호소,"경상북도 예천군 예천읍 충효로 632-84 (예천읍, 유기동물보호소)",경상북도 예천군,공고중,1,0,0
434127,경남-의령-2021-00085,개,골든 리트리버,갈색,미상,미상,사람 손 잘 탐,2021-06-22,NaN,경남 의령군 지정면 보덕로 1길 23,2021-06-22 ~ 2021-07-02,하영효,경상남도 의령군 의령읍 남강로 485-75,경상남도 의령군,공고중,2,0,0
434521,경북-청송-2021-00054,개,믹스견,검정+갈색,미상,미상,유순하고 사람을 잘따름,2021-06-23,마을 주변을 돌아다님,청송군 진보면 광덕세장길(광덕1리),2021-06-23 ~ 2021-06-28,점곡보호소,경상북도 청송군 현동면 청송로 3128-135 (현동면),경상북도 청송군,공고중,2,0,0
434522,경북-청송-2021-00055,개,믹스견,흰색,미상,미상,사람을 경계함,2021-06-23,도로에 돌아다님,청송군 청송읍 금곡리(이레타운),2021-06-23 ~ 2021-06-28,점곡보호소,경상북도 청송군 현동면 청송로 3128-135 (현동면),경상북도 청송군,공고중,1,0,0


In [ ]:
df.head()

,공고번호,축종,품종,털색,성별,중성화여부,특징,접수일시,구조사유,발생장소,공고기간,관할보호센터명,보호장소,관할기관,상태,성격,건강,나이
0,경북-예천-2018-00003,개,믹스견,NaN,암컷,아니오,NaN,2018-01-01,NaN,예천읍 통명리,2018-01-03 ~ 2018-01-17,예천군유기동물보호소,"경상북도 예천군 예천읍 충효로 632-84 (예천읍, 유기동물보호소)",경상북도 예천군,NaN,0,0,0
1,광주-남구-2018-00001,개,믹스견,흰색,수컷,미상,"18-0002콩지,다리부상",2018-01-01,NaN,남구 송하동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(안락사),0,2,0
2,광주-남구-2018-00002,개,믹스견,흰색,암컷,미상,18-0003콩이,2018-01-01,NaN,남구 송하동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(입양),0,0,0
3,광주-광산-2018-00001,개,보스턴 테리어,검흰,수컷,아니오,"18-0001볼타,스파이크 목줄착용",2018-01-01,NaN,광산구 하남동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 광산구,종료(자연사),0,0,0
4,경기-수원-2018-00001,개,믹스견,갈색,암컷,아니오,"교통사고 = 골반 / 척추 분리 골절 , 대퇴골두 골절 = 진통제 처치",2018-01-01,NaN,세류사거리,2018-01-01 ~ 2018-01-08,광교산동물병원,경기도 수원시 장안구 영화동 127-39,경기도 수원시,종료(안락사),0,2,0


In [ ]:
# # 모든 컬럼 유니크 개수, 값들
# for i in df.columns:
#   print(i)
#   print(len(df[i].unique()))
#   print(df[i].unique())

공고번호
429910
['경북-예천-2018-00003' '광주-남구-2018-00001' '광주-남구-2018-00002' ...
 '전남-광양-2021-00251' '제주-제주-2021-02423' '서울-성북-2021-00094']
축종
2
['개' '고양이']
품종
706
['믹스견' '보스턴 테리어' '푸들' '포메라니안' '한국 고양이' '말티즈' '시츄' '라브라도 리트리버' '알라스칸 말라뮤트'
 '비숑 프리제' '스피츠' '요크셔 테리어' '골든 리트리버' '진도견' '사모예드' '달마시안' '시바' '닥스훈트'
 '터키시 앙고라' '바센지' '셔틀랜드 쉽독' '웰시 코기 펨브로크' '도베르만' '아키다' '슈나우져'
 '페르시안-페르시안 친칠라' '리트리버' '빠삐용' '발바리' '프렌치 불독' '보더 콜리' '핏불테리어' '페키니즈' '코숏'
 '러시안 블루' '포인터' '미니어쳐 핀셔' '치와와' '시베리안 허스키' '사냥개' '기타' '코카 스파니엘'
 '아메리칸 코카 스파니엘' '고양이' nan '믹스 고양이' '믹스 강아지' '그레이트 덴' '브리타니 스파니엘' '혼합'
 '하일랜드 폴드' '비글' '차우차우' '웰시 테리어' '사모예드믹스' '퍼그' '캐벌리어 킹 찰스 스파니엘' '배회고양이'
 '말라뮤트' '믹스' '샴' '셰퍼드' '삽살개' '맹크스' '미니어쳐 푸들' '풍산견' '흰색' '그레이트 피레니즈' '스핑크스'
 '이탈리안 그레이 하운드' '그레이 하운드' '스탠다드 푸들' '길고양이' '브리티시 쇼트헤어' '샤페이' '사냥개믹스견' '불독'
 '호피' '라이카' '방갈믹스' '미텔 스피츠' '아비시니안' '저먼 셰퍼드 독' '도사' '베들링턴 테리어' '아메리칸 쇼트헤어'
 '케인 코르소' '페르시안 익스트림' '대형견 믹스' '미니어쳐 슈나우저' '아이리쉬 세터' '쇼콧' '포피츠 추정' '콜리'
 '아메리칸불리' '오스트랄리안 셰퍼드 독' '야생고양이' '잭 러셀 테리어' '테리어종믹스' '중대형견' '동경견'


In [ ]:
# df['털색'].unique()

array([nan, '흰색', '검흰', ..., '검정색+검정색', '연갈색,검은회색', '흰바탕 검색반점'],
      dtype=object)

In [ ]:
# df.drop(['Unnamed: 0.1','종료(입양)','종료(안락사)','종료(자연사)',	'종료(반환)',	'종료(기증)',	'종료(방사)',	'종료(미포획)',	'보호중',	'공고중'],axis=1,inplace=True)
# df

,공고번호,축종,품종,털색,성별,중성화여부,특징,접수일시,구조사유,발생장소,공고기간,관할보호센터명,보호장소,관할기관,상태
0,경북-예천-2018-00003,개,믹스견,NaN,암컷,아니오,NaN,2018-01-01,NaN,예천읍 통명리,2018-01-03 ~ 2018-01-17,예천군유기동물보호소,"경상북도 예천군 예천읍 충효로 632-84 (예천읍, 유기동물보호소)",경상북도 예천군,NaN
1,광주-남구-2018-00001,개,믹스견,흰색,수컷,미상,"18-0002콩지,다리부상",2018-01-01,NaN,남구 송하동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(안락사)
2,광주-남구-2018-00002,개,믹스견,흰색,암컷,미상,18-0003콩이,2018-01-01,NaN,남구 송하동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(입양)
3,광주-광산-2018-00001,개,보스턴 테리어,검흰,수컷,아니오,"18-0001볼타,스파이크 목줄착용",2018-01-01,NaN,광산구 하남동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 광산구,종료(자연사)
4,경기-수원-2018-00001,개,믹스견,갈색,암컷,아니오,"교통사고 = 골반 / 척추 분리 골절 , 대퇴골두 골절 = 진통제 처치",2018-01-01,NaN,세류사거리,2018-01-01 ~ 2018-01-08,광교산동물병원,경기도 수원시 장안구 영화동 127-39,경기도 수원시,종료(안락사)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434702,제주-제주-2021-02431,고양이,한국 고양이,흰/검,수컷,예,(개체관리번호21335)두부교상,2021-06-23,NaN,제주시 이도일동 중앙로25길22-1,2021-06-24 ~ 2021-07-05,제주 동물보호센터,제주특별자치도 제주시 첨단동길 184-14 (용강동),제주특별자치도,공고중
434703,서울-성동-2021-00073,개,믹스견,검/흰,수컷,아니오,양귀처짐. 코검정. 배빵빵함. 소심함. 꼬리단미안됨. 털상태양호.,2021-06-23,NaN,행당동 168-151 왕십리역 인근,2021-06-24 ~ 2021-07-05,한국동물구조관리협회,경기도 양주시 남면 감악산로 63-37 (남면),서울특별시 성동구,공고중
434704,전남-광양-2021-00251,개,믹스견,갈색+검정,암컷,아니오,믹스 갈색+검정 암컷 3개월추정,2021-06-23,유기동물,은장도길35-7 주변,2021-06-23 ~ 2021-07-05,광양시 임시보호소,전라남도 광양시 봉강면 인덕로 1169-20 (봉강면) 지곡리 864-24,전라남도 광양시,공고중
434705,제주-제주-2021-02423,고양이,한국 고양이,검정,수컷,아니오,(개체관리번호21327)생후2개월,2021-06-23,NaN,서귀포시 남원읍 위미해안로129번길31-4,2021-06-24 ~ 2021-07-05,제주 동물보호센터,제주특별자치도 제주시 첨단동길 184-14 (용강동),제주특별자치도,공고중


In [ ]:
# '공고기간'(str) -> 수치형 데이터의 기간/날짜로 변환

def datediff(d):
    dates = d.split('~')
    date1 = dates[0].strip()
    date2 = dates[1].strip()
    
    date1 = dt.strptime(date1, "%Y-%m-%d")
    date2 = dt.strptime(date2, "%Y-%m-%d")
    
    date_diff = date2 - date1
    return date_diff.days
    
df['공고기간'] = df['공고기간'].map(datediff)
# '접수일시'(str) -> 데이트 타입 변환
df['접수일시'] = pd.to_datetime(df['접수일시'], infer_datetime_format=True) 

In [ ]:
# df[:50]['특징']

0                                         NaN
1                              18-0002콩지,다리부상
2                                   18-0003콩이
3                         18-0001볼타,스파이크 목줄착용
4     교통사고 = 골반 / 척추 분리 골절 , 대퇴골두 골절 = 진통제 처치
5                                  연두색옷착용.귀염증
6                            빨간색자동차그림옷, 초록색몸줄
7                        교통사고 당함. 상태 예후 극히 불량
8                        피부병심함. 왼쪽귀 끝 작은 혹있음.
9                               중성화 안되어있고 순함.
10                                          -
11                              뒷다리못씀. 예후극히불량
12                                         패딩
13                                 기력없음, 얼굴다침
14       습진이랑 피부병치료, 접종1차까지 맞춤, 신고자분이 30일 보호함
15                                        온순함
16                                         양호
17                                         순함
18                                         순함
19                                         순함
20                                         순함
21                                

In [ ]:
# 축종 
# 개 / 고양이.. 
# 기타 날릴 것
delete_lst = []
for idx, sp in enumerate(df['축종']):
  if sp == '기타축종':
    delete_lst.append(idx)
df.drop(delete_lst,inplace=True)  

# 품종
# 개 - 기타
df['품종'].unique()
df[df['축종']== '개']['품종'].unique()
# 고양이 - 한국고양이
# df['품종'].unique()
# df[df['축종']== '고양이']['품종'].unique()


# 성별
# 암컷 0
# 수컷 1
# 미상 2 
arr = []
for idx,s in enumerate(df['성별']):
  if s == '암컷':
    arr.append(0)
  elif s == '수컷':
    arr.append(1)
  else:
    arr.append(2)
df['gender'] = arr
gender = ['암컷','수컷','미상']
# 중성화여부
# 예 0
# 아니면 1
# 미상 2
arr = []
for idx,s in enumerate(df['중성화여부']):
  if s == '예':
    arr.append(0)
  elif s == '아니요':
    arr.append(1)
  else:
    arr.append(2)
df['neutering'] = arr
neutering = ['예','아니요','미상']
# 털색
# 유사 값 처리. 


In [ ]:
# # 색깔 단어 분리
# # Okt, Hannanum, Kkma,Komoran, Mecab
# tokenizer = Okt()
# color_dict = {}

# for color in df['털색'].unique():
#   if type(color) == str:
#     print(color,' -> ',end=' ')
#     temp = tokenizer.nouns(color)
#     for c in temp:
#       if c not in color_dict.keys():
#         color_dict[c] = 1
#       else:
#         color_dict[c] = color_dict[c] + 1
#     print(temp)
#   else:
#     print(color)
# # for color in df['털색'].unique():
# #   print(color)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
크림색/주황색/연갈색(귀)  ->  ['크림', '색', '주황색', '갈색', '귀']
크림색/연노란색/갈색(귀)  ->  ['크림', '색', '노란색', '갈색', '귀']
붉은색/흰색  ->  ['은색', '흰색']
회색.흰색.연갈색  ->  ['회색', '흰색', '갈색']
오목천동 청구1차 아파트  ->  ['오목천동', '청구', '차', '아파트']
서귀포시 막숙포로146  ->  ['서귀포시', '막숙포']
흰+연베이지  ->  ['베이지']
흰색 4마리  ->  ['흰색', '마리']
가좌도서관  ->  ['좌', '도서관']
갈색 흰색 믹스  ->  ['갈색', '흰색', '믹스']
황색, 갈색, 검은색, 흰색  ->  ['황색', '갈색', '검은색', '흰색']
5.4  ->  []
쵸코-하양  ->  ['쵸코', '하양']
갈색 겸은색  ->  ['갈색', '겸', '은색']
금&흰색  ->  ['금', '흰색']
크림색/검회색  ->  ['크림', '색', '검', '회색']
군북면 현포로 205-1  ->  ['군북면', '포로']
갈색, 콧등으로흰줄무늬  ->  ['갈색', '콧등', '줄무늬']
연갈색+크림색  ->  ['갈색', '크림', '색']
흰색,연갈색,검은회색  ->  ['흰색', '갈색', '검', '은회색']
힌색+검정+갈색  ->  ['힌색', '검정', '갈색']
곡성읍 읍내25길  ->  ['곡성읍', '읍내', '길']
구례군 산동면 탑동마을회관  ->  ['구례군', '산동면', '탑동', '마을', '회관']
갈색 화이으  ->  ['갈색', '화이']
연한갈색(귀 진한갈색)  ->  ['연한', '갈색', '귀', '갈색']
아이보리, 흰색, 갈색, 검은색 등  ->  ['아이보리', '흰색', '갈색', '검은색', '등']
흰색에미색  ->  ['흰색', '색']
흰색에갈색무늬  ->  ['흰색', '갈색', '무늬']
와동799-17

In [ ]:
# 색 분류
# color_dict = sorted(color_dict.items(),key=lambda c : c[1],reverse=True)
# color_dict
color = {
    '흰색':['흰색','크림','햐양','흰','흰샥','백색','화이트','하양','하얀색','희색','벡','벡색','벡섹','백구','힌색','횐색','휜색','횐','흰섹','힁색','힁','흰샛','화이','배색','진도견'],
    '검정색':['검정','검덩','검졍','검정색','검은색','흑','블랙','흑색','검색','흑갈색','다크','검은빛','검황','검회','검점','까만색','검혼','불랙','검절','겅','검종'],
    '갈색':['갈색','걀색','고둥','갈','브라운','연가','밤색','고동색','초코','고동','밤색','초콜릿','쵸코','흙','흙색','갈섹','암갈색','진밤','커피','초코렛','다갈색','칡색','걸색','초콜렛','동색','강색','칡','갈샥','회걀색','간색'],
    '회색':['회색','진회색','그레이','재색','실버','연회색','쥐색','쥐','잿빛','진회','회색빛','회','회갈색','은색','은회색','은','회검','은빛','길색','연회','잿색','철색','러시안블루','차콜'],
    '황색':['황색','환색','확색','황','황토색','황토','담황색','황백색','보리','누런빛','누런색','누런','누렁','누렁색','진도','황검','회황색','항색','황회','황섹','황새'],
    '노랑색':['노랑','노런','노랑색','노','노란색','연노랑','노라','옐로','노란빛','레몬','연노','진노','옐로우','노검','노혼','노란섹','노랭','누렇색','치즈'],
    '주황색':['주황색','주황','누렁이','오렌지색','오렌지','주','연주황색','연주황','호랑','붉은색','레드','적색','빨간색','홍','붉은빛','홍색','빨','빨강','정색','레즈'],
    '금색':['금색','골드','골든','순금','금','금은','황금','금빛'],
    '살구색':['살구색','살색','살구','분홍','분홍색','핑크','핑크색','연분홍'],
    '고등어':['고등어','고등'],
    '베이지':['베이지','베이지색','베이직'],
    '아이보리':['아이보리','크림'],
    '삼색':['삼색'],
    '턱시도':['턱','시도'],
    '카오스':['카오스']
}
# color_dict

In [ ]:
tokenizer = Okt()
color_feat = []
for cr in df['털색']:
  color_sum = ''
  if isinstance(cr,str):
    temp = tokenizer.nouns(cr)
    if len(temp) == 0:
      color_sum += 'unknwon'
    for c in temp:
      for key in color.keys():
        if c in color[key]:
          color_sum += key + ' '
    if color_sum == '':
      color_sum += 'unknown'
  else:
    color_sum += 'unknown'
  color_feat.append(color_sum.strip())  
df['털색2'] = color_feat

In [ ]:
print(len(color_feat))
print(len(set(color_feat)))
print(set(color_feat))


429927
643
{'황색 살구색', '검정색 회색 고등어 흰색 턱시도 턱시도', '검정색 회색 흰색', '황색 갈색 갈색 검정색', '흰색 회색 흰색 아이보리 갈색', '카오스', '회색 황색 흰색', '흰색 회색 갈색', '갈색 금색', '노랑색 노랑색 흰색', '주황색 회색 흰색 아이보리', '황색 갈색 검정색 흰색', '흰색 갈색 살구색', '삼색 검정색 흰색 갈색', '노랑색 검정색 흰색 아이보리', '황색 흰색 갈색 검정색', '갈색 고등어', '흰색 검정색 노랑색 흰색', '검정색 금색 갈색', '회색 흰색 검정색 고등어', '주황색 검정색 갈색 흰색', '금색 검정색', '회색 아이보리 황색', '갈색 흰색 주황색', '검정색 황색 회색', '살구색 회색', '삼색 흰색 황색', '베이지 흰색', '흰색 갈색 회색', '황색 회색 검정색', '흰색 갈색 회색 삼색', '갈색 턱시도', '검정색 회색 회색', '검정색 흰색 갈색 흰색', '흰색 노랑색', '살구색 흰색 아이보리', '흰색 검정색 흰색 아이보리', '베이지 주황색 갈색', 'unknown', '황색 아이보리 회색', '갈색 흰색 아이보리 검정색', '베이지 검정색', '흰색 고등어 턱시도 턱시도', '갈색 검정색 회색', '턱시도 흰색', '노랑색 흰색', '검정색 흰색 주황색', '흰색 아이보리 베이지', '고등어 흰색 아이보리', '고등어 금색', '흰색 아이보리', '흰색 주황색 갈색 갈색', '흰색 황색 황색', '삼색 흰색 노랑색 검정색', '베이지 갈색 검정색', '갈색 베이지 검정색', '갈색 베이지 흰색', '흰색 검정색 회색 고등어', '노랑색 회색 흰색', '회색 흰색 갈색', '황색 회색', '노랑색 검정색 흰색', '삼색 노랑색', '흰색 노랑색 살구색', '흰색 금색', '검정색 턱시도 흰색', '노랑색 카오스 고등어', '회색 회색 흰색', '베이지 회색', '갈색 주황색 검정색', '흰색 주황색 흰색 아이보리', '흰색 검정색 흰색', '노랑색 검정색 갈

In [ ]:
# okt = Okt() 

# word_dict = {}
# for dog_breed in df[df['축종']=='개']['품종']:
#   if type(dog_breed) == str:
#     print(dog_breed,' -> ',end=' ')
#     temp = okt.nouns(dog_breed)
#     for db in temp:
#       if db not in word_dict.keys():
#         word_dict[db] = 1
#       else:
#         word_dict[db] = word_dict[db] + 1
#     print(temp)
#   else:
#     print(dog_breed)

# word_dict = sorted(word_dict.items(),key=lambda db : db[1], reverse=True)


# c_word_dict = {}
# for cat_breed in df[df['축종']=='고양이']['품종']:
#   if type(cat_breed) == str:
#     print(cat_breed,' -> ',end=' ')
#     temp2 = okt.nouns(cat_breed)
#     for cb in temp2:
#       if cb not in c_word_dict.keys():
#         c_word_dict[cb] = 1
#       else:
#         c_word_dict[cb] = c_word_dict[cb] + 1
#     print(temp2)
#   else:
#     print(cat_breed)
    
# c_word_dict = sorted(c_word_dict.items(),key=lambda cb : cb[1], reverse=True)

In [ ]:
breed = {
    '믹스견' : ['믹스견','믹스','혼합','기타','잡종','잡종견','들개','재래종','혼종','혼혈'],
    '한국 고양이' : ['고양이','한국','코숏','기타','고등어','치즈','혼종','턱시도','시도','턱','잡종','삼색','태비','젖소','들고양이','코리안','야생고양이','길','모름','코쇽','무늬','콧숏','믹스견','혼','알수없음','추정']
}

def dbreed(data): 
    if data == '푸들':
        return '품종견'
    if isinstance(data, str):
        temp = okt.nouns(data)
        status = 0
        if len(temp)==0:
            return '믹스견'
        
        for i in temp:        
            if i in breed['믹스견']:
                status =1

        if status == 0:
            return '품종견'
        else :
            if len(temp) >= 2:
                return '품종 믹스'
            else:
                return '믹스견'
    else: return '믹스견'
        
def cbreed(data):  
    if isinstance(data, str): # 문자열 일 경우
        temp = okt.nouns(data)
        status = 0
        
        if len(temp)==0:
            return '한국 고양이'
        
        for i in temp:        
            if i in breed['한국 고양이']:
                status =1

        if status == 0:
            return '품종묘'
        else :
            if len(temp) >= 2:
                return '품종 믹스'
            else:
                return '한국 고양이'        
    else: return '한국 고양이'  # 문자열 x
    

In [ ]:
breed_lst=[]
for i in df[['축종','품종']].values:
    if i[0] == '개':
        breed_lst.append(dbreed(i[1]))
    else:
        breed_lst.append(cbreed(i[1]))
        
df['품종2'] = breed_lst

In [ ]:
df.head()

,공고번호,축종,품종,털색,성별,중성화여부,특징,접수일시,구조사유,발생장소,공고기간,관할보호센터명,보호장소,관할기관,상태,성격,건강,나이,gender,neutering,털색2,품종2
0,경북-예천-2018-00003,개,믹스견,NaN,암컷,아니오,NaN,2018-01-01,NaN,예천읍 통명리,2018-01-03 ~ 2018-01-17,예천군유기동물보호소,"경상북도 예천군 예천읍 충효로 632-84 (예천읍, 유기동물보호소)",경상북도 예천군,NaN,0,0,0,0,2,unknown,믹스견
1,광주-남구-2018-00001,개,믹스견,흰색,수컷,미상,"18-0002콩지,다리부상",2018-01-01,NaN,남구 송하동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(안락사),0,2,0,1,2,흰색,믹스견
2,광주-남구-2018-00002,개,믹스견,흰색,암컷,미상,18-0003콩이,2018-01-01,NaN,남구 송하동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(입양),0,0,0,0,2,흰색,믹스견
3,광주-광산-2018-00001,개,보스턴 테리어,검흰,수컷,아니오,"18-0001볼타,스파이크 목줄착용",2018-01-01,NaN,광산구 하남동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 광산구,종료(자연사),0,0,0,1,2,unknown,품종견
4,경기-수원-2018-00001,개,믹스견,갈색,암컷,아니오,"교통사고 = 골반 / 척추 분리 골절 , 대퇴골두 골절 = 진통제 처치",2018-01-01,NaN,세류사거리,2018-01-01 ~ 2018-01-08,광교산동물병원,경기도 수원시 장안구 영화동 127-39,경기도 수원시,종료(안락사),0,2,0,0,2,갈색,믹스견


In [ ]:
# 공고 번호 -> 지역 칼럼 추가
locations = []
for i in df['공고번호']:
  if isinstance(i,str):
    temp = i.split('-')
    locations.append(temp[0]+' '+temp[1])
  else:
    locations.append('unknown')
df['지역'] = locations
df.head()


,공고번호,축종,품종,털색,성별,중성화여부,특징,접수일시,구조사유,발생장소,공고기간,관할보호센터명,보호장소,관할기관,상태,성격,건강,나이,gender,neutering,털색2,품종2,지역
0,경북-예천-2018-00003,개,믹스견,NaN,암컷,아니오,NaN,2018-01-01,NaN,예천읍 통명리,2018-01-03 ~ 2018-01-17,예천군유기동물보호소,"경상북도 예천군 예천읍 충효로 632-84 (예천읍, 유기동물보호소)",경상북도 예천군,NaN,0,0,0,0,2,unknown,믹스견,경북 예천
1,광주-남구-2018-00001,개,믹스견,흰색,수컷,미상,"18-0002콩지,다리부상",2018-01-01,NaN,남구 송하동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(안락사),0,2,0,1,2,흰색,믹스견,광주 남구
2,광주-남구-2018-00002,개,믹스견,흰색,암컷,미상,18-0003콩이,2018-01-01,NaN,남구 송하동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(입양),0,0,0,0,2,흰색,믹스견,광주 남구
3,광주-광산-2018-00001,개,보스턴 테리어,검흰,수컷,아니오,"18-0001볼타,스파이크 목줄착용",2018-01-01,NaN,광산구 하남동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 광산구,종료(자연사),0,0,0,1,2,unknown,품종견,광주 광산
4,경기-수원-2018-00001,개,믹스견,갈색,암컷,아니오,"교통사고 = 골반 / 척추 분리 골절 , 대퇴골두 골절 = 진통제 처치",2018-01-01,NaN,세류사거리,2018-01-01 ~ 2018-01-08,광교산동물병원,경기도 수원시 장안구 영화동 127-39,경기도 수원시,종료(안락사),0,2,0,0,2,갈색,믹스견,경기 수원


In [ ]:
df['상태'].unique()

array([nan, '종료(안락사)', '종료(입양)', '종료(자연사)', '종료(반환)', '종료(방사)', '종료(기증)',
       '종료(미포획)', '보호중', '공고중'], dtype=object)

In [ ]:
# random frost
# linear regression


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 429927 entries, 0 to 434706
Data columns (total 23 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   공고번호       429926 non-null  object
 1   축종         429927 non-null  object
 2   품종         428727 non-null  object
 3   털색         429914 non-null  object
 4   성별         429927 non-null  object
 5   중성화여부      429927 non-null  object
 6   특징         421717 non-null  object
 7   접수일시       429927 non-null  object
 8   구조사유       76626 non-null   object
 9   발생장소       429927 non-null  object
 10  공고기간       429927 non-null  object
 11  관할보호센터명    429926 non-null  object
 12  보호장소       429926 non-null  object
 13  관할기관       429927 non-null  object
 14  상태         421761 non-null  object
 15  성격         429927 non-null  int64 
 16  건강         429927 non-null  int64 
 17  나이         429927 non-null  int64 
 18  gender     429927 non-null  int64 
 19  neutering  429927 non-null  int64 
 20  털색2 

In [ ]:
# 공고번호 빈거 하나 날리기
df.drop(index=df[df['공고번호'].isna()].index ,axis=0,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 429926 entries, 0 to 434706
Data columns (total 23 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   공고번호       429926 non-null  object
 1   축종         429926 non-null  object
 2   품종         428726 non-null  object
 3   털색         429913 non-null  object
 4   성별         429926 non-null  object
 5   중성화여부      429926 non-null  object
 6   특징         421716 non-null  object
 7   접수일시       429926 non-null  object
 8   구조사유       76626 non-null   object
 9   발생장소       429926 non-null  object
 10  공고기간       429926 non-null  object
 11  관할보호센터명    429925 non-null  object
 12  보호장소       429925 non-null  object
 13  관할기관       429926 non-null  object
 14  상태         421761 non-null  object
 15  성격         429926 non-null  int64 
 16  건강         429926 non-null  int64 
 17  나이         429926 non-null  int64 
 18  gender     429926 non-null  int64 
 19  neutering  429926 non-null  int64 
 20  털색2 

In [ ]:
# 잘못 크롤링 되어 비어버린 상태 값 날리기
df.drop(index=df[df['상태'].isna()].index, axis=0,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421761 entries, 1 to 434706
Data columns (total 23 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   공고번호       421761 non-null  object
 1   축종         421761 non-null  object
 2   품종         420565 non-null  object
 3   털색         421757 non-null  object
 4   성별         421761 non-null  object
 5   중성화여부      421761 non-null  object
 6   특징         421716 non-null  object
 7   접수일시       421761 non-null  object
 8   구조사유       76611 non-null   object
 9   발생장소       421761 non-null  object
 10  공고기간       421761 non-null  object
 11  관할보호센터명    421760 non-null  object
 12  보호장소       421760 non-null  object
 13  관할기관       421761 non-null  object
 14  상태         421761 non-null  object
 15  성격         421761 non-null  int64 
 16  건강         421761 non-null  int64 
 17  나이         421761 non-null  int64 
 18  gender     421761 non-null  int64 
 19  neutering  421761 non-null  int64 
 20  털색2 

In [ ]:
df.to_csv('/content/drive/MyDrive/TwoPs/AniHelp_feat3.csv',encoding='utf-8')

## '특징' 데이터 처리


In [ ]:
# 불용어 처리
stop_words = ['']

In [ ]:
# 단어들 (명사, 동사, 형용사) 빈도 보기
okt = Okt()
feat_arr = [] # 
word_dic = {}
for word in df['특징']:
  if isinstance(word, str):
    temp = okt.pos(word, norm=True, stem =True)
    feat = []
    for w in temp:
      if w[1] in ['Noun','Verb','Adjective']:
        feat.append(w[0])
        if not (w[0] in word_dic):
          word_dic[w[0]] = 0
        word_dic[w[0]] += 1
    feat_arr.append(feat)


In [ ]:
for word in feat_arr:
  if ('없다' in word) and ('있다' in word) :
    print(word)

['치석', '없다', '미용', '않다', '상태', '온순하다', '경계', '심보', '있다']
['칩', '없다', '빌라', '앞', '박스', '버려지다', '있다', '함']
['칩', '없다', '빌라', '앞', '박스', '버려지다', '있다', '함']
['왼쪽', '귀가', '없다', '오른쪽', '어깨', '부분', '버즘', '있다']
['교통사고', '겁', '많다', '외상', '없다', '먼지', '흙', '묻다', '기침', '있다', '움']
['마름', '이빨', '없다', '미용', '되다', '배', '수술', '자국', '실밥', '있다']
['오른쪽', '귀', '상처', '있다', '꼬리', '없다']
['털', '관리', '안되다', '순하다', '사람', '없다', '짖다', '있다', '있다']
['배', '혹', '있다', '목줄', '없다', '사람', '따르다']
['기운', '없다', '외상', '흔적', '없다', '발작', '있다', '예후', '불량']
['마이크로', '칩', '없다', '치석', '있다', '왼쪽', '무릎', '뼈', '탈구', '추정', '오른쪽', '눈', '안충', '있다', '구충제', '투여']
['침', '흘리다', '기운', '없다', '쓰러지다', '호흡', '있다']
['꼬리', '끝', '털다', '긺', '귀', '처지다', '사람', '경계', '겁', '있다', '미안', '되다', '털때', '타다', '고령', '슬개골', '기력', '없다', '식욕', '없다', '콧물']
['탈구', '있다', '일상생활', '지장', '없다', '활발', '공고', '기간', '중', '입양', '가능']


In [ ]:
feat_arr

[['콩', '다리부상'],
 ['콩'],
 ['볼타', '스파이크', '목줄', '착용'],
 ['교통사고', '골반', '척추', '분리', '골절', '대퇴골', '골절', '진통제', '처치'],
 ['연두색', '옷', '착용', '염증'],
 ['빨간색', '자동차', '그림', '옷', '초록색', '몸줄'],
 ['교통사고', '당하다', '상태', '예후', '극히', '불량'],
 ['피부병', '심하다', '왼쪽', '귀', '끝', '작다', '혹', '있다'],
 ['중성', '안되다', '순하다'],
 [],
 ['패딩'],
 ['기력', '없다', '얼굴', '다치다'],
 ['습진', '피부병', '치료', '접종', '차', '맞춤', '고자', '보호', '함'],
 ['온순하다'],
 ['양호'],
 ['순하다'],
 ['순하다'],
 ['순하다'],
 ['순하다'],
 [],
 ['어리다', '강아지', '임'],
 ['겁', '많다', '순하다', '움'],
 ['파란', '목줄', '순하다'],
 ['단', '밉다'],
 ['노랗다', '눈꼽', '심하다', '마르다', '편', '윗니', '빠지다'],
 ['초록색', '목줄', '착용', '경계', '함'],
 ['노랑', '색줄', '들어가다', '초록색', '목줄', '착용', '경계', '함'],
 ['목줄', '없다', '온순하다'],
 ['은색', '체인', '목걸이', '착용', '어리다', '강아지', '임'],
 ['파랑색', '목줄', '착용', '온순하다'],
 ['목줄', '없다', '온순하다'],
 ['파란', '목줄', '가슴', '희다', '털'],
 ['개월', '추정'],
 ['빨강', '색', '목줄', '착용', '온순하다'],
 ['목줄', '없다', '체', '구가', '작다', '온순하다'],
 ['목줄', '없다', '비숑프리제', '믹스견', '보임', '온순하다'],
 ['목줄', '없다', '사람', '따르다'],
 ['옷'

In [ ]:
# 빈도가 높은 순 정렬
word_dic = sorted(word_dic.items(), key= lambda item: item[1],reverse=True)
word_dic

[('순하다', 34475),
 ('개월', 33722),
 ('사람', 31140),
 ('관리', 29150),
 ('없다', 28972),
 ('추정', 28911),
 ('목줄', 28329),
 ('생후', 27355),
 ('상태', 26717),
 ('착용', 26123),
 ('온순하다', 24546),
 ('번호', 24442),
 ('되다', 24439),
 ('털', 23394),
 ('개체', 23258),
 ('있다', 22853),
 ('코', 19039),
 ('마리', 18023),
 ('많다', 16968),
 ('겁', 16950),
 ('심하다', 15553),
 ('갈색', 15415),
 ('양호', 15250),
 ('흰색', 13854),
 ('꼬리', 13373),
 ('검정', 13349),
 ('구조', 12985),
 ('어미', 12728),
 ('함', 12273),
 ('따름', 11717),
 ('움', 11521),
 ('개', 11495),
 ('강아지', 10965),
 ('미용', 10835),
 ('경계', 10790),
 ('호', 10243),
 ('교통사고', 9891),
 ('눈', 9861),
 ('발견', 9811),
 ('좋아하다', 9730),
 ('기력', 9727),
 ('월령', 9407),
 ('매우', 9127),
 ('얌전하다', 9000),
 ('눈곱', 8327),
 ('새끼', 8207),
 ('불량', 8062),
 ('미안', 7584),
 ('하다', 7471),
 ('온순', 7469),
 ('칩', 7396),
 ('피부병', 7164),
 ('활발하다', 7130),
 ('귀', 7060),
 ('따르다', 7012),
 ('타다', 6874),
 ('아이', 6709),
 ('검정색', 6554),
 ('계심', 6483),
 ('어리다', 6421),
 ('옷', 6379),
 ('약간', 6349),
 ('허약', 6099),
 ('자다', 5918)

In [ ]:
len(word_dic)

17382

In [ ]:
# 빈도 20개 초과인 단어
for i in word_dic:
  if i[1] > 20:
    print(i[0],i[1])

순하다 34475
개월 33722
사람 31140
관리 29150
없다 28972
추정 28911
목줄 28329
생후 27355
상태 26717
착용 26123
온순하다 24546
번호 24442
되다 24439
털 23394
개체 23258
있다 22853
코 19039
마리 18023
많다 16968
겁 16950
심하다 15553
갈색 15415
양호 15250
흰색 13854
꼬리 13373
검정 13349
구조 12985
어미 12728
함 12273
따름 11717
움 11521
개 11495
강아지 10965
미용 10835
경계 10790
호 10243
교통사고 9891
눈 9861
발견 9811
좋아하다 9730
기력 9727
월령 9407
매우 9127
얌전하다 9000
눈곱 8327
새끼 8207
불량 8062
미안 7584
하다 7471
온순 7469
칩 7396
피부병 7164
활발하다 7130
귀 7060
따르다 7012
타다 6874
아이 6709
검정색 6554
계심 6483
어리다 6421
옷 6379
약간 6349
허약 6099
자다 5918
되어다 5835
건강 5793
탈진 5720
유기 5714
자국 5607
고양이 5557
사나 5545
눈물 5538
밉다 5276
줄 5267
믹스견 5207
공고 5056
목걸이 5028
뒷다리 4958
피부질환 4950
콧물 4919
상처 4859
단 4720
결막염 4706
신고 4705
성격 4664
골절 4622
저하 4554
빨갛다 4504
털때 4453
믹스 4430
질환 4326
입소 4287
소심하다 4274
안되다 4269
보호 4144
설사 4041
쫑긋 3988
치다 3987
희다 3963
견 3909
기립 3894
탈수 3756
박스 3684
사상충 3563
분홍 3562
두 3472
가슴 3466
접힘 3453
후지 3414
젖먹이 3411
진드기 3356
오염 3338
안 3325
않다 3300
빨간색 3286
치석 3256
포획 3239
얼굴 3230
다리 

In [ ]:
# 특징 feature를 특정 카테고리로 분류하기 위함
character_ct = { # -1  +2
    1 : ['경계','계심','겁','사나','소심하다','의심','짖다','공격','사납다','공격성',
         '으르렁','무섭다','불안','신경','힘없다','겁쟁이','산만하다','불안하다','낯','약간'],
    2 : ['좋아하다','붙임성','사람','순하다','온순하다','얌전하다','활발하다','따르다','좋다','착하다',
         '애교','영리하다','밝다','친화력','활달함','명랑하다','친해지다','차분하다','조용하다',
         '활달함','에너지','점잖다','순진하다','따름','친근하다', '착하다']
}
health_ct = {    # -1  --2
    1 : ['빨갛다','미흡','불량','여위다','결막염','혼탁','기력','허약','쇄약','눈곱','오염','오물',
         '저하','설사','탈수','마름','영양실조','허약하다','눈꼽심함','힘없다','구취','노란눈꼽','악취',
         '학대','쇠약하다','눈꼽기','눈꼽많음','저체중','부실','말랐슴','침','흘리다','저하','설사',
         '탈수','진드기','접종','백신','예방','장염','구내염','콧물심함','경미하다'],
    2 : ['상태','부다','심하다','교상','불능','결막염','혼탁','빠지다','털','교통사고','심각','상처',
         '골절','피부','질병','마비','염증','양호','피부병','척추','뒷다리','부정교합','이염', '심장',
         '피부질환','탈장','회충','절단','골절','질환','사상충','피부','백내장','출혈','안좋다','부상',
         '사고','다리부상','앞다리','탈모','치료','내장','이상','다치다','뒷발','증상','손상','잃다','수술',
         '나쁘다','소실','종양','질병','구더기','눈병','구강','괴사','전염','장애','기생충','돌출',
         '광견병','각막','복부','기관지염','절단','아프다','불편','불량하다','흉터','파열','궤양','바이러스',
         '실명','구충','하반신마비','뼈','혈변','기형','감염증','찢어지다','폐렴','귓병','슬개골탈구',
         '고름','거동','모낭충','벗겨지다','척추골절','아래턱','전염병','안염','혼수상태','교통','코마',
         '발작','통증','회충','짖무름','허피스바이러스','쇼크','절뚝거림','회충증','황달','응급치료','패혈증',
         '부실','내출혈','탈모증','진물','낙상','각막염','녹내장','종기','중상']
}
age_ct = {
    1: ['개월','월령','개체','관리','어리다','새끼','생후','강아지','아가','젖먹이','나이','애기','새끼고양이','이유식','분유','꼬물이','젖','갓난']
}

In [ ]:
# 특징 feature로 부터 새로운 데이터 파생
character = np.zeros(df.shape[0],dtype=np.int16)
health = np.zeros(df.shape[0],dtype=np.int16)
age = np.zeros(df.shape[0],dtype=np.int16)

okt = Okt()

for idx,word in enumerate(df['특징']):
  if isinstance(word, str):
    temp = okt.pos(word, norm=True, stem =True)
  
    for w in temp:
      if w[1] in ['Noun','Verb','Adjective']:
        for c in character_ct: # - 1, + 2
          if w[0] in character_ct[c]:
            if (c== 1) and ('없다' in temp):
              character[idx] = 2
            character[idx] = c
        for h in health_ct:
          if w[0] in health_ct[h]:
            health[idx] = h
        for a in age_ct:
          if w[0] in age_ct[a]:
            age[idx] = a  
        

d = {
    '성격':character,
    '건강':health,
    '나이':age
}
df2 = pd.DataFrame(d)


In [ ]:
# df['성격'] = df2['성격']
# df['건강'] = df2['건강']
# df['나이'] = df2['나이']
df.head()

,공고번호,축종,품종,털색,성별,중성화여부,특징,접수일시,구조사유,발생장소,공고기간,관할보호센터명,보호장소,관할기관,상태,성격,건강,나이,gender,neutering,털색2,품종2,지역
1,광주-남구-2018-00001,개,믹스견,흰색,수컷,미상,"18-0002콩지,다리부상",2018-01-01,NaN,남구 송하동,10,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(안락사),0.0,0.0,0.0,1,2,흰색,믹스견,광주 남구
2,광주-남구-2018-00002,개,믹스견,흰색,암컷,미상,18-0003콩이,2018-01-01,NaN,남구 송하동,10,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(입양),0.0,0.0,0.0,0,2,흰색,믹스견,광주 남구
3,광주-광산-2018-00001,개,보스턴 테리어,검흰,수컷,아니오,"18-0001볼타,스파이크 목줄착용",2018-01-01,NaN,광산구 하남동,10,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 광산구,종료(자연사),0.0,2.0,0.0,1,2,unknown,품종견,광주 광산
4,경기-수원-2018-00001,개,믹스견,갈색,암컷,아니오,"교통사고 = 골반 / 척추 분리 골절 , 대퇴골두 골절 = 진통제 처치",2018-01-01,NaN,세류사거리,7,광교산동물병원,경기도 수원시 장안구 영화동 127-39,경기도 수원시,종료(안락사),0.0,2.0,0.0,0,2,갈색,믹스견,경기 수원
5,경기-성남-2018-00001,개,푸들,흑색,수컷,아니오,연두색옷착용.귀염증,2018-01-01,유기견발견후 신고,중원도서관근처,10,펫토피아동물병원,"경기도 성남시 분당구 불정로 266 (수내동, 유신제일조합)",경기도 성남시,종료(반환),0.0,0.0,0.0,1,2,검정색,품종견,경기 성남


In [ ]:
# 파생된 특징 dataframe 합치기
# df = pd.concat([df,df2], axis=1)
# df

,공고번호,축종,품종,털색,성별,중성화여부,특징,접수일시,구조사유,발생장소,공고기간,관할보호센터명,보호장소,관할기관,상태,성격,건강,나이
0,경북-예천-2018-00003,개,믹스견,NaN,암컷,아니오,NaN,2018-01-01,NaN,예천읍 통명리,2018-01-03 ~ 2018-01-17,예천군유기동물보호소,"경상북도 예천군 예천읍 충효로 632-84 (예천읍, 유기동물보호소)",경상북도 예천군,NaN,0,0,0
1,광주-남구-2018-00001,개,믹스견,흰색,수컷,미상,"18-0002콩지,다리부상",2018-01-01,NaN,남구 송하동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(안락사),0,2,0
2,광주-남구-2018-00002,개,믹스견,흰색,암컷,미상,18-0003콩이,2018-01-01,NaN,남구 송하동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(입양),0,0,0
3,광주-광산-2018-00001,개,보스턴 테리어,검흰,수컷,아니오,"18-0001볼타,스파이크 목줄착용",2018-01-01,NaN,광산구 하남동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 광산구,종료(자연사),0,0,0
4,경기-수원-2018-00001,개,믹스견,갈색,암컷,아니오,"교통사고 = 골반 / 척추 분리 골절 , 대퇴골두 골절 = 진통제 처치",2018-01-01,NaN,세류사거리,2018-01-01 ~ 2018-01-08,광교산동물병원,경기도 수원시 장안구 영화동 127-39,경기도 수원시,종료(안락사),0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434702,제주-제주-2021-02431,고양이,한국 고양이,흰/검,수컷,예,(개체관리번호21335)두부교상,2021-06-23,NaN,제주시 이도일동 중앙로25길22-1,2021-06-24 ~ 2021-07-05,제주 동물보호센터,제주특별자치도 제주시 첨단동길 184-14 (용강동),제주특별자치도,공고중,0,0,1
434703,서울-성동-2021-00073,개,믹스견,검/흰,수컷,아니오,양귀처짐. 코검정. 배빵빵함. 소심함. 꼬리단미안됨. 털상태양호.,2021-06-23,NaN,행당동 168-151 왕십리역 인근,2021-06-24 ~ 2021-07-05,한국동물구조관리협회,경기도 양주시 남면 감악산로 63-37 (남면),서울특별시 성동구,공고중,1,2,0
434704,전남-광양-2021-00251,개,믹스견,갈색+검정,암컷,아니오,믹스 갈색+검정 암컷 3개월추정,2021-06-23,유기동물,은장도길35-7 주변,2021-06-23 ~ 2021-07-05,광양시 임시보호소,전라남도 광양시 봉강면 인덕로 1169-20 (봉강면) 지곡리 864-24,전라남도 광양시,공고중,0,0,1
434705,제주-제주-2021-02423,고양이,한국 고양이,검정,수컷,아니오,(개체관리번호21327)생후2개월,2021-06-23,NaN,서귀포시 남원읍 위미해안로129번길31-4,2021-06-24 ~ 2021-07-05,제주 동물보호센터,제주특별자치도 제주시 첨단동길 184-14 (용강동),제주특별자치도,공고중,0,0,1


In [ ]:
df.to_csv('/content/drive/MyDrive/AniHelp_feat4.csv',encoding='utf-8')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421761 entries, 1 to 434706
Data columns (total 23 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   공고번호       421761 non-null  object        
 1   축종         421761 non-null  object        
 2   품종         420565 non-null  object        
 3   털색         421757 non-null  object        
 4   성별         421761 non-null  object        
 5   중성화여부      421761 non-null  object        
 6   특징         421716 non-null  object        
 7   접수일시       421761 non-null  datetime64[ns]
 8   구조사유       76611 non-null   object        
 9   발생장소       421761 non-null  object        
 10  공고기간       421761 non-null  int64         
 11  관할보호센터명    421760 non-null  object        
 12  보호장소       421760 non-null  object        
 13  관할기관       421761 non-null  object        
 14  상태         421761 non-null  object        
 15  성격         409050 non-null  float64       
 16  건강         409050 no

### 특징을 잠재 의미를 분석하는 토픽 모델에 적용
##### 특징들을 토픽별로 잘 모으는 것 같지 않음

In [ ]:
# LDA ( 잠재 디클레리 분석) 을 위한 전처리
# 단어들을 토큰화하여 LDA 모델 입력 준비
okt = Okt()
st_arr = [] # 

for word in df['특징']:
  if isinstance(word, str):
    temp = okt.pos(word, norm=True, stem =True)
    sentence = []
    for w in temp:
      if w[1] in ['Noun','Verb','Adjective']:
        sentence.append(w[0])
    st_arr.append(sentence)


In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(st_arr) # (word_id, word_frequency) 의 형태를 만들기 위해 단어 사전 생성
corpus = [dictionary.doc2bow(text) for text in st_arr] # 각 문장당 (word_id, word_frequency) 형태로 인코딩
print(corpus[1]) # 수행된 결과에서 두번째 문장 출력. 첫번째 문서의 인덱스는 0

[(1, 1)]


In [ ]:
# LDA 모델 학습
import gensim
NUM_TOPICS = 10 #20개의 토픽, k=20
ldamodel2 = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15) # passes는 알고리즘의 동작 횟수, 
topics2 = ldamodel.print_topics(num_words=30) # num_words=4로 총 4개의 단어만 출력 
for topic in topics2:
    print(topic) 

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(El

(0, '0.194*"개월" + 0.163*"생후" + 0.138*"추정" + 0.135*"관리" + 0.109*"번호" + 0.100*"개체" + 0.046*"계심" + 0.025*"사상충" + 0.024*"안" + 0.013*"심장" + 0.008*"강하다" + 0.004*"털엉킴" + 0.003*"상자" + 0.002*"발목" + 0.002*"연두색" + 0.002*"가다" + 0.002*"견주" + 0.001*"소유자" + 0.001*"인수" + 0.001*"하트" + 0.001*"낯선" + 0.001*"관찰" + 0.001*"사이" + 0.001*"결막" + 0.001*"파랑" + 0.001*"들어가다" + 0.001*"파출소" + 0.001*"불안" + 0.001*"이주" + 0.001*"집앞"')
(1, '0.134*"심하다" + 0.102*"입소" + 0.067*"피부병" + 0.058*"유기" + 0.051*"박스" + 0.036*"접종" + 0.033*"필요하다" + 0.029*"번" + 0.023*"치료" + 0.022*"버려지다" + 0.021*"예쁘다" + 0.021*"담기다" + 0.020*"친" + 0.017*"필요" + 0.017*"적" + 0.013*"병원" + 0.013*"묘" + 0.012*"사납다" + 0.009*"구충" + 0.008*"악질" + 0.008*"넣다" + 0.006*"힘" + 0.006*"공격성" + 0.006*"달반" + 0.006*"종이" + 0.005*"옆" + 0.005*"바람" + 0.005*"고자" + 0.005*"응원" + 0.005*"케이"')
(2, '0.100*"교통사고" + 0.058*"골절" + 0.052*"뒷다리" + 0.040*"후지" + 0.037*"및" + 0.036*"추정" + 0.026*"등록" + 0.024*"예후" + 0.022*"불량" + 0.022*"내장" + 0.021*"염증" + 0.019*"출혈" + 0.019*"우측" + 0.017*"이상" + 0.016*"동시"

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [ ]:
# 주제가 10개
topics2 = ldamodel.print_topics(num_words=100) # num_words=30로 총 30개의 단어만 출력 
for topic in topics2:
    print(topic) 

(0, '0.194*"개월" + 0.163*"생후" + 0.138*"추정" + 0.135*"관리" + 0.109*"번호" + 0.100*"개체" + 0.046*"계심" + 0.025*"사상충" + 0.024*"안" + 0.013*"심장" + 0.008*"강하다" + 0.004*"털엉킴" + 0.003*"상자" + 0.002*"발목" + 0.002*"연두색" + 0.002*"가다" + 0.002*"견주" + 0.001*"소유자" + 0.001*"인수" + 0.001*"하트" + 0.001*"낯선" + 0.001*"관찰" + 0.001*"사이" + 0.001*"결막" + 0.001*"파랑" + 0.001*"들어가다" + 0.001*"파출소" + 0.001*"불안" + 0.001*"이주" + 0.001*"집앞" + 0.001*"성대수술" + 0.001*"생리" + 0.001*"몸매" + 0.000*"검색" + 0.000*"기르다" + 0.000*"안과" + 0.000*"쇄약" + 0.000*"시장" + 0.000*"파목줄" + 0.000*"노목줄" + 0.000*"순딩" + 0.000*"앵무새" + 0.000*"후분" + 0.000*"청평" + 0.000*"무상처" + 0.000*"원피스" + 0.000*"홍일점" + 0.000*"요추" + 0.000*"녹목줄" + 0.000*"올인원" + 0.000*"카키목줄" + 0.000*"남양" + 0.000*"초목" + 0.000*"김화" + 0.000*"군복" + 0.000*"둘" + 0.000*"뱀" + 0.000*"개성" + 0.000*"남움" + 0.000*"다시다" + 0.000*"노색목줄" + 0.000*"변산면" + 0.000*"탄탄하다" + 0.000*"컨" + 0.000*"목즐" + 0.000*"빨파목줄" + 0.000*"봉인" + 0.000*"주홍" + 0.000*"잘생겻음" + 0.000*"색줄" + 0.000*"좌천" + 0.000*"우수관" + 0.000*"관릴" + 0.000*"등외" + 0.000

In [ ]:
# 주제가 20개
topics = ldamodel.print_topics(num_words=20) # num_words=4로 총 4개의 단어만 출력 
for topic in topics:
    print(topic) 

(0, '0.194*"개월" + 0.163*"생후" + 0.138*"추정" + 0.135*"관리" + 0.109*"번호" + 0.100*"개체" + 0.046*"계심" + 0.025*"사상충" + 0.024*"안" + 0.013*"심장" + 0.008*"강하다" + 0.004*"털엉킴" + 0.003*"상자" + 0.002*"발목" + 0.002*"연두색" + 0.002*"가다" + 0.002*"견주" + 0.001*"소유자" + 0.001*"인수" + 0.001*"하트"')
(1, '0.134*"심하다" + 0.102*"입소" + 0.067*"피부병" + 0.058*"유기" + 0.051*"박스" + 0.036*"접종" + 0.033*"필요하다" + 0.029*"번" + 0.023*"치료" + 0.022*"버려지다" + 0.021*"예쁘다" + 0.021*"담기다" + 0.020*"친" + 0.017*"필요" + 0.017*"적" + 0.013*"병원" + 0.013*"묘" + 0.012*"사납다" + 0.009*"구충" + 0.008*"악질"')
(2, '0.100*"교통사고" + 0.058*"골절" + 0.052*"뒷다리" + 0.040*"후지" + 0.037*"및" + 0.036*"추정" + 0.026*"등록" + 0.024*"예후" + 0.022*"불량" + 0.022*"내장" + 0.021*"염증" + 0.019*"출혈" + 0.019*"우측" + 0.017*"이상" + 0.016*"동시" + 0.016*"연제구" + 0.015*"안구" + 0.014*"마비" + 0.014*"낑낑대다" + 0.013*"좌측"')
(3, '0.349*"없다" + 0.107*"기력" + 0.090*"칩" + 0.062*"설사" + 0.054*"영양" + 0.047*"생기다" + 0.032*"체온" + 0.029*"애교" + 0.028*"주일" + 0.023*"포획틀" + 0.020*"금정" + 0.010*"주황색" + 0.009*"라인" + 0.008*"특징" + 0.

In [ ]:
# LDA 시각화 툴 설치
!pip install pyLDAvis

     |████████████████████████████████| 1.7MB 3.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7MB 182kB/s 
     |████████████████████████████████| 9.9MB 36.3MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-cp37-none-any.whl size=136897 sha256=1ed97063a75b07a6800e59ba5b634b5b4af461114c31f9df0c2a1d7491c269e1
  Stored in directory: /root/.cache/pip/wheels/a0/9c/fc/c6e00689d35c82cf96a8adc70edfe7ba7904374fdac3240ac2
Successfully built pyLDAvis
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.21.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


ModuleNotFoundError: ignored

In [ ]:
# 크롬 개발자 도구 console 창에 입력하면 90분 뒤 아무 조작없음이 안뜨도록 계속 클릭을 해줌
# function ClickConnect() {
#     var buttons = document.querySelectorAll("colab-dialog.yes-no-dialog paper-button#cancel"); 
#     buttons.forEach(function(btn) { 
#         btn.click(); 
#     }); 
#     console.log("1분마다 자동 재연결"); 
#     document.querySelector("colab-toolbar-button#connect").click(); 
# } 
# setInterval(ClickConnect,1000*60);

# 데이터 분석

##### 데이터 불러오기

In [ ]:
# 정희 데이터 불러오는 부분
jeonghee_path = '/content/drive/MyDrive/TwoPs/AniHelp_feat3.csv'
df = pd.read_csv(jeonghee_path, index_col=0)
df.head()

,공고번호,축종,품종,털색,성별,중성화여부,특징,접수일시,구조사유,발생장소,공고기간,관할보호센터명,보호장소,관할기관,상태,성격,건강,나이,gender,neutering,털색2,품종2,지역
1,광주-남구-2018-00001,개,믹스견,흰색,수컷,미상,"18-0002콩지,다리부상",2018-01-01,NaN,남구 송하동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(안락사),0,2,0,1,2,흰색,믹스견,광주 남구
2,광주-남구-2018-00002,개,믹스견,흰색,암컷,미상,18-0003콩이,2018-01-01,NaN,남구 송하동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(입양),0,0,0,0,2,흰색,믹스견,광주 남구
3,광주-광산-2018-00001,개,보스턴 테리어,검흰,수컷,아니오,"18-0001볼타,스파이크 목줄착용",2018-01-01,NaN,광산구 하남동,2018-01-01 ~ 2018-01-11,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 광산구,종료(자연사),0,0,0,1,2,unknown,품종견,광주 광산
4,경기-수원-2018-00001,개,믹스견,갈색,암컷,아니오,"교통사고 = 골반 / 척추 분리 골절 , 대퇴골두 골절 = 진통제 처치",2018-01-01,NaN,세류사거리,2018-01-01 ~ 2018-01-08,광교산동물병원,경기도 수원시 장안구 영화동 127-39,경기도 수원시,종료(안락사),0,2,0,0,2,갈색,믹스견,경기 수원
5,경기-성남-2018-00001,개,푸들,흑색,수컷,아니오,연두색옷착용.귀염증,2018-01-01,유기견발견후 신고,중원도서관근처,2018-01-01 ~ 2018-01-11,펫토피아동물병원,"경기도 성남시 분당구 불정로 266 (수내동, 유신제일조합)",경기도 성남시,종료(반환),0,2,0,1,2,검정색,품종견,경기 성남


In [ ]:
df.head(40)

,공고번호,축종,품종,털색,성별,중성화여부,특징,접수일시,구조사유,발생장소,공고기간,관할보호센터명,보호장소,관할기관,상태,성격,건강,나이,gender,neutering,털색2,품종2,지역
1,광주-남구-2018-00001,개,믹스견,흰색,수컷,미상,"18-0002콩지,다리부상",2018-01-01,NaN,남구 송하동,10,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(안락사),0,2,0,1,2,흰색,믹스견,광주 남구
2,광주-남구-2018-00002,개,믹스견,흰색,암컷,미상,18-0003콩이,2018-01-01,NaN,남구 송하동,10,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 남구,종료(입양),0,0,0,0,2,흰색,믹스견,광주 남구
3,광주-광산-2018-00001,개,보스턴 테리어,검흰,수컷,아니오,"18-0001볼타,스파이크 목줄착용",2018-01-01,NaN,광산구 하남동,10,광주 동물보호소,"광주광역시 북구 본촌마을길 25 (본촌동, 건국동사무소) 광주 동물보호소",광주광역시 광산구,종료(자연사),0,0,0,1,2,unknown,품종견,광주 광산
4,경기-수원-2018-00001,개,믹스견,갈색,암컷,아니오,"교통사고 = 골반 / 척추 분리 골절 , 대퇴골두 골절 = 진통제 처치",2018-01-01,NaN,세류사거리,7,광교산동물병원,경기도 수원시 장안구 영화동 127-39,경기도 수원시,종료(안락사),0,2,0,0,2,갈색,믹스견,경기 수원
5,경기-성남-2018-00001,개,푸들,흑색,수컷,아니오,연두색옷착용.귀염증,2018-01-01,유기견발견후 신고,중원도서관근처,10,펫토피아동물병원,"경기도 성남시 분당구 불정로 266 (수내동, 유신제일조합)",경기도 성남시,종료(반환),0,2,0,1,2,검정색,품종견,경기 성남
6,경기-안양-2018-00001,개,포메라니안,갈,수컷,예,"빨간색자동차그림옷, 초록색몸줄",2018-01-01,NaN,동안구청,12,한국야생동물보호협회,경기도 안산시 상록구 청곡길 50 (부곡동),경기도 안양시,종료(입양),0,0,0,1,0,unknwon,품종견,경기 안양
7,경기-부천-2018-00001,고양이,한국 고양이,"흰,갈,검",암컷,미상,교통사고 당함. 상태 예후 극히 불량,2018-01-01,도로변에 교통사고 당한 고양이가 있어 신고,경기도 부천시 춘의동 93,10,24시아이동물메디컬,경기도 부천시 오정구 원종동 229-8,경기도 부천시,종료(자연사),0,1,0,0,2,unknown,품종 믹스,경기 부천
8,경기-부천-2018-00002,개,푸들,갈,암컷,미상,피부병심함. 왼쪽귀 끝 작은 혹있음.,2018-01-01,길거리에서 배회중인것을 구조,119소방구조대,10,24시아이동물메디컬,경기도 부천시 오정구 원종동 229-8,경기도 부천시,종료(자연사),0,2,0,0,2,unknwon,품종견,경기 부천
9,경기-부천-2018-00003,개,믹스견,갈,수컷,아니오,중성화 안되어있고 순함.,2018-01-01,오정대공원에서 발견 집까지 따라옴,오정대공원,10,24시아이동물메디컬,경기도 부천시 오정구 원종동 229-8,경기도 부천시,종료(자연사),2,0,0,1,2,unknwon,믹스견,경기 부천
10,대구-수성-2018-00001,개,믹스견,흰색,수컷,예,-,2018-01-01,NaN,수성구청당직실,10,제니스동물병원,대구광역시 수성구 달구벌대로 2475 (범어동),대구광역시 수성구,종료(반환),0,0,0,1,0,흰색,믹스견,대구 수성


#### 1차 가설

1. 축종 : 개/고양이 3:1
  - 개가 높나 고양이가 높나(입양률 비교)

1. 품종 : 믹스/품종믹스/품종/모름
  - 개는 소형견이 
  - 고양이는 품종묘가 더 높을것
1. 성별 : 암/수/모름
  - 품종에 한해서? 암컷높을 것
1. 중성화여부: y/n/모름
  - 크게 영향 안미칠 것 같
1. 털색 : (주관식)
  - 흰색이 높을 것 같고, 그 외에는 특이한 무늬가 높을 것(달마시안)
1. 연도별 : 입양여부
  - 연도별 in/out되는 축종 비교 해보면 고양이의 in이 많아질 것. 
  - 소가족 비율, 자취생 비율, 
  - 사회구조 변화 등 집에서 활동률 높아짐. (2020-21)
  - 평균 공고 기간의 변화
1. 지역 
  - 지역별 income되는 숫자
  - 지역별 outcome되는 숫자
  - 지역별 공고 기간
1. 특징 : 성격/건강/나이
  - 성격: 사람 친화적인 아이들이 잘 나갈 것
  - 건강: 큰 질병이 없는 아이들
  - 나이: (고양이)어릴 수록 잘 나갈 거라 생각됨, 
1. 공고기간 
  - 7.10.12.14 정도로 값이 있던데, 길수록 (공고기간 지나서도 보호중인 아이들도 있음) 입양률 높아질 것 >> 
 